##### Price Extraction

Develop an approach that will find the numerical value of the price from any comment.
Prices may be indicated in local currency or US dollars.

The solution leverages pre-trained multilanguage model `Babelscape/wikineural-multilingual-ner` to classify strings that might contain prices. The final extraction of price is done by `price_parser` library.

In [ ]:
# Libraries setup
import os
import re
import string
import unicodedata
import nltk
import torch

import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from price_parser import Price

load_dotenv()

# Setup pytorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Download the NLTK resource
nltk.download('stopwords')
nltk.download('punkt')

In [19]:
# Define paths
fpath_data = os.environ.get("FPATH_DATA")

# Load reviews data
reviews = pd.read_csv(fpath_data+"order_reviews.csv")

In [20]:
# It looks like all reviews are from Brazil according to customers.csv

# Convert to datetime column review_creation_date
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])

# For the sake of simplicit, lets merge review comment title and message
reviews['review_concat'] = reviews['review_comment_title'].fillna('')+', '+reviews['review_comment_message'].fillna('')

# # Drop rows with null ['review_concat']
# reviews = reviews.dropna(subset=['review_concat'])

# Drop rows with only ', ' string
reviews = reviews[reviews.review_concat != ', ']

# Lets keep only relevant columns
reviews = reviews[['review_id', 'order_id', 'review_concat','review_creation_date']]

##### Preprocessing

In [21]:
# Lets check if there are any numbers in the reviews - this will be helpfull for the second task
def search_numbers(text):
    return bool(re.search(r'\d+', text))

reviews['contain_num'] = reviews['review_concat'].apply(search_numbers)
# reviews[reviews['contain_num']==True].head(50)

In [22]:
def remove_emojis(text):
    """Remove emojis from the text."""
    if isinstance(text, str):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)
    else:
        return text

In [23]:
def keep_only_ascii(text):
    def is_supported(char):
        try:
            char.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return False
        else:
            return True
        
    def clean_string(s):
        return ''.join(c for c in s if is_supported(c) or unicodedata.category(c) not in ['Cn', 'Co', 'Cs'])

    cleaned_list = [clean_string(s) for s in text]
    return [s for s in cleaned_list if s]  # Remove any empty strings

In [24]:
def remove_stopwords(tokens):
    """Eliminate common stopwords from the tokenized text.
    Stop words are commonly used words like "the," "is," or 
    "and" that don't carry much meaning and can be removed to reduce noise in the data
    """
    # stop_words = set(stopwords.words('english')) # English version - we will anyway use PT as the translation will be done after preprocessing
    stop_words = set(stopwords.words('portuguese')) # PT version
    return [word for word in tokens if word not in stop_words]

In [25]:
def tokenize_text(text):
    """Split the text into individual words or tokens."""
    return word_tokenize(text)

In [26]:
def remove_numbers(text):
    """Exclude numerical digits from the text."""
    return re.sub(r'\d+', '', text)

def remove_punctuation(text):
    """Remove punctuation marks from the text."""
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_extra_whitespaces(text):
    """Remove extra white space from text."""
    return re.sub(r'\s+', ' ', text, flags=re.I)

def remove_single_chars(text):
    """Remove all single characters from text"""
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

def remove_special_chars(text):
    """Remove all the special characters from text."""
    return re.sub(r'\W', ' ', text)

def remove_not_alphabetical(text):
    """Remove any character that isn't alphabetical."""
    return re.sub(r'[^a-zA-Z\s]', '', text)

In [27]:
# Final preprocessing pipeline, some of the functions might not be used in the solution, althrough defined
def preprocess(text):
    text = str(text).lower() # To lower
    text = remove_extra_whitespaces(text)
    # text = remove_numbers(text)
    text = remove_punctuation(text)
    text = tokenize_text(text)
    text = remove_stopwords(text)
    text = remove_emojis(text)
    text = keep_only_ascii(text)
    return text

In [28]:
reviews['reviews_processed'] = reviews['review_concat'].apply(preprocess)
reviews['reviews_processed_str'] = reviews['reviews_processed'].apply(' '.join)

In [29]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner").to(device)

e:\MiniConda\envs\vala\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
# Load a pre-trained NER model
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True, device=device)

def extract_price(text):
    # Use NER to identify monetary entities (MISC)
    entities = nlp(text)
    return entities

e:\MiniConda\envs\vala\lib\site-packages\transformers\pipelines\token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [31]:
tqdm.pandas(desc="Extracting prices")
reviews['extracted_price'] = reviews['reviews_processed_str'].progress_apply(extract_price)

Extracting prices:   0%|          | 8/43482 [00:00<21:47, 33.25it/s]e:\MiniConda\envs\vala\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Extracting prices: 100%|██████████| 43482/43482 [12:16<00:00, 59.03it/s]


,review_id,order_id,review_concat,review_creation_date,contain_num,reviews_processed,reviews_processed_str,extracted_price
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,", Recebi bem antes do prazo estipulado.",2017-04-21,False,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,[]
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,", Parabéns lojas lannister adorei comprar pela...",2018-03-01,False,"[parabéns, lojas, lannister, adorei, comprar, ...",parabéns lojas lannister adorei comprar intern...,[]
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,"recomendo, aparelho eficiente. no site a marca...",2018-05-22,True,"[recomendo, aparelho, eficiente, site, marca, ...",recomendo aparelho eficiente site marca aparel...,"[{'entity_group': 'MISC', 'score': 0.55567026,..."
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,", Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16,False,"[pouco, travandopelo, valor, ta, boa]",pouco travandopelo valor ta boa,[]
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,"Super recomendo, Vendedor confiável, produto o...",2018-05-23,False,"[super, recomendo, vendedor, confiável, produt...",super recomendo vendedor confiável produto ok ...,"[{'entity_group': 'MISC', 'score': 0.43961123,..."
...,...,...,...,...,...,...,...,...
99983,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,", Entregou dentro do prazo. O produto chegou e...",2018-02-07,False,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,[]
99990,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,", O produto não foi enviado com NF, não existe...",2018-05-19,False,"[produto, enviado, nf, existe, venda, nf, cert...",produto enviado nf existe venda nf certeza fic...,[]
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,", Excelente mochila, entrega super rápida. Sup...",2018-03-22,False,"[excelente, mochila, entrega, super, rápida, s...",excelente mochila entrega super rápida super r...,[]
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,", Solicitei a compra de uma capa de retrovisor...",2017-12-15,False,"[solicitei, compra, capa, retrovisor, celtapri...",solicitei compra capa retrovisor celtaprismame...,"[{'entity_group': 'MISC', 'score': 0.39793238,..."


In [91]:
def extract_entity_groups(extracted_price_str):
    try:
        entity_groups = [item['entity_group'] for item in extracted_price_str]
        entity_groups = ' '.join(str(x) for x in entity_groups)
        return entity_groups
    except:
        return None

In [92]:
reviews['entity_groups'] = reviews['extracted_price'].apply(extract_entity_groups)
reviews['entity_groups'].unique()

array(['', 'MISC ORG', 'MISC', 'MISC MISC', 'LOC LOC', 'MISC PER', 'PER',
       'MISC MISC MISC', 'ORG', 'MISC MISC PER', 'MISC PER MISC', 'LOC',
       'MISC MISC ORG', 'MISC LOC', 'MISC PER MISC PER', 'LOC MISC',
       'PER PER', 'PER LOC MISC', 'PER MISC', 'PER PER PER',
       'LOC LOC LOC LOC', 'MISC PER ORG', 'MISC MISC ORG ORG',
       'PER PER MISC', 'PER MISC PER MISC', 'ORG ORG', 'PER ORG',
       'MISC ORG PER', 'MISC LOC MISC', 'MISC PER MISC MISC', 'ORG PER',
       'LOC PER', 'MISC PER PER MISC', 'LOC LOC LOC', 'PER MISC PER',
       'MISC MISC MISC MISC', 'MISC ORG MISC', 'PER LOC', 'MISC PER PER'],
      dtype=object)

In [119]:
def extract_price_from_misc(text, entity_group):
    if 'MISC' not in entity_group:
        return None
   
    # Use regex to find price mentions
    regex_prices = re.findall(r'(?:R\$|\$|€|£|\bUSD|\bEUR|\bGBP|\bBRL|\bVES)\s?(\d+(?:[\.,]\d+)?)|(\d+(?:[\.,]\d+)?)\s?(?:reais|dollars?|euros?|pounds?|bolivares?)', text, flags=re.IGNORECASE)    
    
    numeric_prices = []
    for price_tuple in regex_prices:
        price_str = price_tuple[0] if price_tuple[0] else price_tuple[1]
        if price_str:
            try:
                price_obj = Price.fromstring(price_str)
                if price_obj.amount is not None:
                    numeric_prices.append(price_obj)
            except ValueError:
                continue
   
    return numeric_prices if numeric_prices else None

def process_row(row):
    text = row['reviews_processed_str']
    entity_group = row['entity_groups']
    return extract_price_from_misc(text, entity_group)

In [121]:
reviews['found_price'] = reviews.apply(process_row, axis=1)

,review_id,order_id,review_concat,review_creation_date,contain_num,reviews_processed,reviews_processed_str,extracted_price,len_extracted_price,entity_groups,len_entity_groups,found_price
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,", Recebi bem antes do prazo estipulado.",2017-04-21,False,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,[],0,,0,None
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,", Parabéns lojas lannister adorei comprar pela...",2018-03-01,False,"[parabéns, lojas, lannister, adorei, comprar, ...",parabéns lojas lannister adorei comprar intern...,[],0,,0,None
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,"recomendo, aparelho eficiente. no site a marca...",2018-05-22,True,"[recomendo, aparelho, eficiente, site, marca, ...",recomendo aparelho eficiente site marca aparel...,"[{'entity_group': 'MISC', 'score': 0.55567026,...",2,MISC ORG,8,None
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,", Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16,False,"[pouco, travandopelo, valor, ta, boa]",pouco travandopelo valor ta boa,[],0,,0,None
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,"Super recomendo, Vendedor confiável, produto o...",2018-05-23,False,"[super, recomendo, vendedor, confiável, produt...",super recomendo vendedor confiável produto ok ...,"[{'entity_group': 'MISC', 'score': 0.43961123,...",1,MISC,4,None
...,...,...,...,...,...,...,...,...,...,...,...,...
99983,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,", Entregou dentro do prazo. O produto chegou e...",2018-02-07,False,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,[],0,,0,None
99990,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,", O produto não foi enviado com NF, não existe...",2018-05-19,False,"[produto, enviado, nf, existe, venda, nf, cert...",produto enviado nf existe venda nf certeza fic...,[],0,,0,None
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,", Excelente mochila, entrega super rápida. Sup...",2018-03-22,False,"[excelente, mochila, entrega, super, rápida, s...",excelente mochila entrega super rápida super r...,[],0,,0,None
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,", Solicitei a compra de uma capa de retrovisor...",2017-12-15,False,"[solicitei, compra, capa, retrovisor, celtapri...",solicitei compra capa retrovisor celtaprismame...,"[{'entity_group': 'MISC', 'score': 0.39793238,...",1,MISC,4,None


In [122]:
reviews[reviews['found_price'].notnull()]

,review_id,order_id,review_concat,review_creation_date,contain_num,reviews_processed,reviews_processed_str,extracted_price,len_extracted_price,entity_groups,len_entity_groups,found_price
3349,591b2149765d02a35621930a4800ceb9,eb0393138e2e8b7164947608489f069e,", Nem a nota foi emitida. Cansei e solicitei a...",2017-11-09,True,"[nota, emitida, cansei, solicitei, stark, canc...",nota emitida cansei solicitei stark cancelamen...,"[{'entity_group': 'MISC', 'score': 0.51398975,...",1,MISC,4,"[Price(amount=Decimal('100'), currency=None)]"
22058,c8aa80ce5db16def531c812c22e86373,6f8b0c86311ec8c9e1558eff54965728,"é uma merda , bem comprei um produto que custa...",2018-06-20,True,"[merda, bem, comprei, produto, custava, 26100,...",merda bem comprei produto custava 26100 reais ...,"[{'entity_group': 'MISC', 'score': 0.53748953,...",1,MISC,4,"[Price(amount=Decimal('26100'), currency=None)..."
47844,a745fa7cc29b37f745b71368ba569063,38541d08d4eb7d571b5b80cb6ca9be03,", Péssima loja, cobrou mais de 20 reais de fre...",2017-12-22,True,"[péssima, loja, cobrou, 20, reais, frete, faz,...",péssima loja cobrou 20 reais frete faz mês fiz...,"[{'entity_group': 'MISC', 'score': 0.54695064,...",1,MISC,4,"[Price(amount=Decimal('20'), currency=None)]"
59783,8c2530791b2510a4b8f474cc52f49847,752829804c0080d1b395f5c9e8801e39,"Gostei do produto, Produto bom e barato, não f...",2018-05-18,True,"[gostei, produto, produto, bom, barato, fiquem...",gostei produto produto bom barato fiquem esper...,"[{'entity_group': 'MISC', 'score': 0.70535815,...",1,MISC,4,"[Price(amount=Decimal('20'), currency=None)]"
68341,f9892402c5e911cc38c9224d0e1292ce,a4cab55a223b91d0013b1f7d634eff40,", Você compra um mp3 player, para ouvir suas m...",2017-07-20,True,"[compra, mp3, player, ouvir, músicas, mp3, cer...",compra mp3 player ouvir músicas mp3 certo erra...,"[{'entity_group': 'MISC', 'score': 0.6081023, ...",3,MISC MISC MISC,14,"[Price(amount=Decimal('40'), currency=None)]"
79201,ea8bfd711bc50eba25a3f23b1aa30878,62d68905605e2398997788c26b9e0096,"Bom, Gostei do produto, mas dois dias depois.....",2018-08-02,True,"[bom, gostei, produto, dois, dias, 6000, reais...",bom gostei produto dois dias 6000 reais barato😐,"[{'entity_group': 'MISC', 'score': 0.5876827, ...",1,MISC,4,"[Price(amount=Decimal('6000'), currency=None)]"


In [124]:
found_prices = reviews[reviews['found_price'].notnull()]
found_prices.to_csv('found_prices.csv')